The purpose of this notebook is to show the use of LSTMs for processing sequences. 

Specifically, we try to compute the sum of two binay digits,
delegating to the model the task of taking care of the propagation of the carry.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Conv1D, Dense, Lambda
from tensorflow.keras.models import Model
from 

Here is our generator. Each element of the resulting batch is a pair (a,res)
where a[0] and a[1] are two sequences of lenght seqlen of binary digits, and
res is their sum. The digits are supposed to be represented in a positional order with less significative digits at lower positions (left to rigth).

The initial carry of the generator is 0; at successive invocations it 
reuses the final carry of the previous sum.

In [76]:
def generator(batchsize,seqlen):
    init_carry = np.zeros(batchsize)
    carry = init_carry
    while True:
      #print("initial carry = ", carry)
      a = np.random.randint(2,size=(batchsize,seqlen,2))
      res = np.zeros((batchsize,seqlen))
      for t in range(0,seqlen):
        sum = a[:,t,0]+a[:,t,1] + carry
        res[:,t] = sum % 2
        carry = sum // 2
      yield (a, res)

Let's create an instance of the generator.

In [80]:
gen = generator(1,2)

And now let's see a few samples.

In [79]:
a,res = next(gen)
print("a1 = {}, a2={}. res = {}".format(a[0,:,0],a[0,:,1],res[0]))

a1 = [1 0], a2=[1 1]. res = [0. 0.]


We can now define the model. It takes in input a pair of boolean sequences of unspecified length. The batchsize dimension is, as usual, implicit too.

In [147]:
def gen_model():
    xa = Input(shape=(None,2))
    x = Conv1D(8,1,activation='relu')(xa)
    x = Conv1D(4,1,activation='relu')(x)
    #x = xa
    x = LSTM(4,activation=None, return_sequences=True)(x)
    x = Dense(1,activation='sigmoid')(x)
    out = tf.squeeze(x,2)
    #out = x
    comp = Model(inputs=xa, outputs=out)
    return comp

In [148]:
mymodel = gen_model()
mymodel.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, None, 2)]         0         
                                                                 
 conv1d_12 (Conv1D)          (None, None, 8)           24        
                                                                 
 conv1d_13 (Conv1D)          (None, None, 4)           36        
                                                                 
 lstm_12 (LSTM)              (None, None, 4)           144       
                                                                 
 dense_11 (Dense)            (None, None, 1)           5         
                                                                 
 tf.compat.v1.squeeze_7 (TFO  (None, None)             0         
 pLambda)                                                        
                                                           

In [145]:
mymodel.compile(optimizer='adam',loss='mse')

In [142]:
batchsize=100
seqlen=10

In [143]:
#mymodel.load_weights("weights/lstm.hdf5")

In [146]:
mymodel.fit(generator(batchsize,seqlen), steps_per_epoch=100, epochs=100)
#comp.save_weights("weights/lstm.hdf5")

Epoch 1/100
100/100 [==============================] - 2s 7ms/step - loss: 0.2512
Epoch 2/100
100/100 [==============================] - 1s 7ms/step - loss: 0.2500
Epoch 3/100
100/100 [==============================] - 1s 7ms/step - loss: 0.2500
Epoch 4/100
100/100 [==============================] - 1s 7ms/step - loss: 0.2500
Epoch 5/100
100/100 [==============================] - 1s 7ms/step - loss: 0.2499
Epoch 6/100
100/100 [==============================] - 1s 7ms/step - loss: 0.2499
Epoch 7/100
100/100 [==============================] - 1s 7ms/step - loss: 0.2498
Epoch 8/100
100/100 [==============================] - 1s 6ms/step - loss: 0.2496
Epoch 9/100
100/100 [==============================] - 1s 7ms/step - loss: 0.2492
Epoch 10/100
100/100 [==============================] - 1s 7ms/step - loss: 0.2481
Epoch 11/100
100/100 [==============================] - 1s 7ms/step - loss: 0.2451
Epoch 12/100
100/100 [==============================] - 1s 7ms/step - loss: 0.2367
Epoch 13/100


In [138]:
example,res = next(generator(1,10))
predicted = np.array([int(np.rint(x)) for x in mymodel.predict(example)[0]])

print("a1        = {}".format(example[0][:,0].astype(int)))
print("a2        = {}".format(example[0][:,1].astype(int)))
print("expected  = {}".format(res[0].astype(int)))
print("predicted = {}".format(predicted))

1/1 [==============================] - 0s 21ms/step
a1        = [1 0 0 0 1 0 0 0 0 1]
a2        = [0 0 0 1 0 0 0 1 1 0]
expected  = [1 0 0 1 1 0 0 1 1 1]
predicted = [1 1 0 1 1 0 0 1 1 1]


WOW!